<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/PH_TrainTestSplit_Ontology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Declare whether you are on Colab or local
colab = True

In [2]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')

  #set path to data in Google Drive
  data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/All Topics"

  #install required packages
  !pip install duckdb
  #!pip install -U sentence-transformers

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 7.6 MB/s 


In [3]:
# Import sentence_transformers 
# from sentence_transformers import SentenceTransformer, util

In [4]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import duckdb
import sqlalchemy
#%load_ext sql

# Train-validation-test split the ontology

### Load data

We have multiple data sources which we could use to "split" the ontoloogy. 

#### Ontology + synonyms

This list "only" contains the main entity_id and the synonym. Will be relevant as soon as we have a list of main entities which we want to exclude from the train dataset .

In [5]:
if colab==True:
  ontology_syn = pd.read_csv(data_path+"/Synonyms in Ontology/isontology.csv", index_col=0)
else:
  ontology_syn = pd.read_csv("../Data/SynonymsinOntology/isontology.csv", index_col=0)
ontology_syn.shape

(397942, 4)

In [8]:
ontology_syn.head()

,entity_id,category,label,synonym
0,IS topic,domain specific entity,TOPIC,IS topic
1,IS topic,domain specific entity,TOPIC,IS topics
2,IS sourcing,domain specific entity,TOPIC,IS / IT sourcing
3,IS sourcing,domain specific entity,TOPIC,tool sourcing
4,IS sourcing,domain specific entity,TOPIC,software sourcings


##### Downsampling entities with more than 100 synonyms

In [9]:
## Check number of synonyms per entity_id
#ent_sy_group = ontology_syn.groupby("entity_id")[["synonym"]].nunique().sort_values(by="synonym", ascending=False)

In [10]:
# # Downsample entities with more than 100 synonyms to a 100 synoyms (but always include "main entity_id" in subsample)
# idx = ent_sy_group[ent_sy_group["synonym"]>100].index.tolist()

# dfs = []
# for entity in idx:
#   t1 = ontology_syn[(ontology_syn.entity_id==entity)&(ontology_syn.synonym==entity)]
#   t2 = ontology_syn[(ontology_syn.entity_id==entity)&(ontology_syn.synonym!=entity)].sample(99)
#   dfs.append(pd.concat([t1,t2]))
# dfs.append(ontology_syn.loc[~ontology_syn["entity_id"].isin(idx)]) # Append dataframe for all entities with less then 100 synonyms to list

# # Combine all dataframes 
# ontology_syn_ds = pd.concat(dfs).reset_index(drop=True)

In [11]:
#Show synonyms in ontology which appear in more than 1 entity
#ontology_syn_ds[ontology_syn_ds.synonym.duplicated(keep=False)].sort_values(by="synonym")

In [12]:
#List of unique entities from ontology with synonyms with downsampling (apparently there are some duplicates in there?!)
#entities_syn_unique_ds = ontology_syn_ds.synonym.unique().tolist()
#ontology_syn_ds_unique = ontology_syn_ds.drop_duplicates(subset="synonym")

#### Ontology.json (only structure of ontology with "main" entity ids (without synonmys))

I did not manage yet to flatten the json file, therefore I used the entities table from the duckdb instead (see next section) 

In [13]:
if colab==True:
  ontology_json = pd.read_json(data_path+"/Ontology/isontology.json")#, orient="values")#, index_col=0)

In [14]:
ontology_json.head()

,root
0,{'IS topic': [{'IS sourcing': [{'outsourcing':...
1,{'research paradigm': [{'positivism': ['logica...
2,"{'model': ['taxonomy', 'framework', {'research..."
3,"{'economic sector': ['private household', {'go..."
4,{'company type': [{'small and mid-size enterpr...


#### Entities (Sentence_id + tagged entity_id) by Rolands script

In [15]:
# Note that DBeaver must be closed, before you can run this command!
if colab==True:
  con = duckdb.connect(database=data_path+'/Extracted Information of Corpus/isrecon.duckdb', read_only=False)
else:
  con = duckdb.connect(database='../Data/ExtractedInformationofCorpus/isrecon.duckdb', read_only=False)

Lets load all the entities from the duckdb

In [16]:
# Load first article as df
entities = con.execute("""
                       SELECT article_id, sentence_id, ent_id, level_1, level_2, level_3, level_4, level_5, level_6, level_7, level_8, level_9, level_10,
                       FROM entities e
                       """).fetchdf()

entities.shape

(1797740, 13)

In [17]:
# Main entities of first level of ontology
entities.level_1.unique()

array(['domain specific entity', 'theoretical entity',
       'methodological entity'], dtype=object)

In [19]:
# Distribution of entities from database accross first and second level
entities.groupby(["level_1", "level_2"])[["ent_id"]].aggregate(["count", "nunique"])

ent_id        
                                              count nunique
level_1                level_2                             
domain specific entity IS technology         463211     475
                       IS topic              461262     416
                       economic sector       102057     234
                       study object           61799      21
methodological entity  data analysis method   73002     292
                       research method       257820     272
                       sampling                6148      27
                       statistical metric     78673      87
                       validity               11357      25
theoretical entity     level of analysis      12064      13
                       model                 196827      31
                       research paradigm      13429      14
                       theory                 60091     223

## `Train-val-test` split of the existing ontology

In order to determine how well our model/embeddings are in identifing new entities or synonyms for existing entities we have to create a `train-val-test split` version of the existing ontology. Meaning we have to select some entities which will be removed from the ontology for training. 


- We will remove different nodes across all main topics and levels of the ontology to assess how our model is handling with missing nodes from different "levels"
- To determine which nodes to include we used the `ontology.json` file since here the hierarchy is already visually included, so it was easy to define some random nodes (and child nodes)

**We selcted the following entities as part of the validattion-test set:**

|    | level_1                | level_2              | entity_id_to_remove                 |   is_parent_node |
|---:|:-----------------------|:---------------------|:------------------------------------|-----------------:|
|  0 | domain specific entity | IS topic             | business process outsourcing        |                1 |
|  1 | domain specific entity | IS topic             | cybersecurity behavior              |                1 |
|  2 | domain specific entity | IS topic             | user interface                      |                1 |
|  3 | domain specific entity | IS topic             | IT planning                         |                1 |
|  4 | domain specific entity | IS topic             | business process management         |                1 |
|  5 | domain specific entity | IS topic             | sharing platform                    |                1 |
|  6 | domain specific entity | IS topic             | software license                    |                1 |
|  7 | domain specific entity | IS topic             | IT recruiting                       |                0 |
|  8 | domain specific entity | IS technology        | enterprise service bus              |                0 |
|  9 | domain specific entity | IS technology        | online marketing                    |                1 |
| 10 | domain specific entity | IS technology        | video game                          |                1 |
| 11 | domain specific entity | IS technology        | non volatile memory                 |                1 |
| 12 | domain specific entity | IS technology        | recommender system                  |                1 |
| 13 | domain specific entity | IS technology        | digital asset management            |                0 |
| 14 | domain specific entity | IS technology        | high level language                 |                1 |
| 15 | domain specific entity | Economic sector      | forest industry                     |                1 |
| 16 | domain specific entity | Economic sector      | hedge fund                          |                0 |
| 17 | domain specific entity | Economic sector      | structural engineering              |                1 |
| 18 | domain specific entity | Economic sector      | research institute                  |                0 |
| 19 | domain specific entity | Economic sector      | private household                   |                0 |
| 20 | domain specific entity | Economic sector      | plastics industry                   |                1 |
| 21 | domain specific entity | study object         | crowdsourcing of participant        |                1 |
| 22 | domain specific entity | study object         | startup                             |                0 |
| 23 | methodological entity  | study object         | microenterprise                     |                0 |
| 24 | methodological entity  | research method      | pilot experiment                    |                0 |
| 25 | methodological entity  | research method      | completely randomized design        |                0 |
| 26 | methodological entity  | research method      | survey design problem               |                1 |
| 27 | methodological entity  | research method      | action research                     |                1 |
| 28 | methodological entity  | research method      | delphi study                        |                0 |
| 29 | methodological entity  | research method      | Markov model                        |                1 |
| 30 | methodological entity  | research method      | database modeling                   |                1 |
| 31 | methodological entity  | data analysis method | constant comparative analysis       |                0 |
| 32 | methodological entity  | data analysis method | ensemble learning                   |                1 |
| 33 | methodological entity  | data analysis method | sentiment analysis                  |                1 |
| 34 | methodological entity  | data analysis method | autoregressive moving average model |                0 |
| 35 | methodological entity  | data analysis method | correlation direction               |                1 |
| 36 | methodological entity  | data analysis method | recurrent neural network            |                1 |
| 37 | methodological entity  | data analysis method | exploratory factor analysis         |                0 |
| 38 | methodological entity  | data analysis method | confirmatory factor analysis        |                0 |
| 39 | methodological entity  | data analysis method | survival function                   |                1 |
| 40 | methodological entity  | data analysis method | p hacking                           |                0 |
| 41 | methodological entity  | statistical metric   | Chi square metric                   |                0 |
| 42 | methodological entity  | statistical metric   | confusion matrix                    |                0 |
| 43 | methodological entity  | validity             | formative validity                  |                0 |
| 44 | methodological entity  | validity             | criterion validity                  |                1 |
| 45 | methodological entity  | sampling             | oversampling                        |                1 |
| 46 | theoretical entity     | sampling             | quota sampling                      |                0 |
| 47 | theoretical entity     | level of analysis    | micro level                         |                1 |
| 48 | theoretical entity     | level of analysis    | country level                       |                0 |
| 49 | theoretical entity     | model                | causal model                        |                0 |
| 50 | theoretical entity     | model                | control variable                    |                0 |
| 51 | theoretical entity     | model                | taxonomy                            |                0 |
| 52 | theoretical entity     | model                | theoretical link                    |                1 |
| 53 | theoretical entity     | research paradigm    | interpretivism                      |                0 |
| 54 | theoretical entity     | research paradigm    | realism                             |                1 |
| 55 | theoretical entity     | theory               | IT security theory                  |                1 |
| 56 | theoretical entity     | theory               | theory of task technology fit       |                0 |
| 57 | theoretical entity     | theory               | personality theory                  |                1 |
| 58 | theoretical entity     | theory               | equity theory                       |                0 |
| 59 | theoretical entity     | theory               | theory of work design               |                1 |
| 60 | theoretical entity     | theory               | personality theory                  |                1 |                 |                0 |
| 61 | theoretical entity     | theory               | uncanny valley theory               |                0 |

We also stored the list above in an Excel format for further processing

In [78]:
data_path_group = data_path[:-10]+"Topic 1/Data_Team1/" # create new data path to access files created by Team1

entities_to_removed = pd.read_excel(data_path_group+"ent_to_remove_from_ontology.xlsx") #load excel list as dataframe
entities_to_removed["entity_id_to_remove"] = entities_to_removed.entity_id_to_remove.str.strip() #remove whitespace

In [79]:
entities_to_removed.head(2)

,level_1,level_2,entity_id_to_remove,is_parent_node
0,domain specific entity,IS topic,business process outsourcing,1.0
1,domain specific entity,IS topic,cybersecurity behavior,1.0


In [80]:
# Create list of entites which should be removed
ent_to_rmv_list = entities_to_removed.entity_id_to_remove.to_list()

Now we want to enhance this dataframe to also include: 
- a list of different entities which belong to the specific parent node
- a list of sentences in which the entity occurs
- a list of articles in which the entity occurs
- also we want the number of occurences for all three points from above

In [81]:
#entities[entities.values == "social rank theory"][["ent_id", "article_id", "sentence_id"]]

In [ ]:
tmp_list=[] # List of entities and child nodes which have to be removed from train ontology 

for entity in ent_to_rmv_list:
  
  #print(entity, (entities.values == entity).any(1).astype(int).max()) #to visually check if the random picked nodes are even included in entities database (so if there has been a match yet)

  tmp_ = entities[entities.values == entity][["ent_id", "article_id", "sentence_id"]].apply(lambda x: x.unique().tolist()) #get all the unique values for all three columns
  df_tmp = pd.DataFrame(tmp_).T #store results as df'
  df_tmp.columns=["ent_list", "articles_list", "sentences_list"] #rename columns
  tmp_list.append(df_tmp) #add df to list

# Add list of children per entity, list of articles and sentences as new column to dataframe
entities_to_removed = pd.concat((entities_to_removed, pd.concat(tmp_list).reset_index(drop=True)), axis=1)

# add number of occurences as separate columns
entities_to_removed[["num_ent", "num_articles", "num_sentences"]] = entities_to_removed[["ent_list", "articles_list", "sentences_list"]].applymap(lambda x: len(x))

In [83]:
entities_to_removed.sample(5)

,level_1,level_2,entity_id_to_remove,is_parent_node,ent_list,articles_list,sentences_list,num_ent,num_articles,num_sentences
53,theoretical entity,research paradigm,interpretivism,0.0,[interpretivism],"[4249, 5607, 862, 1018, 5942, 5943, 4250, 4251...","[4249_17_46, 4249_668_732, 4249_732_818, 4249_...",1,604,2275
31,methodological entity,data analysis method,constant comparative analysis,0.0,[constant comparative analysis],"[864, 2470, 2269, 2270, 2271, 6491, 6493, 2119...","[864_5070_5096, 864_5842_5867, 2470_5181_5222,...",1,130,300
12,domain specific entity,IS technology,recommender system,1.0,"[recommender system, collaborative filtering, ...","[1838, 5414, 5415, 5604, 1623, 2656, 2657, 439...","[1838_2291_2339, 5414_13785_13803, 5415_10502_...",3,311,2680
47,theoretical entity,level of analysis,micro level,1.0,"[individual level, micro level, group level]","[4244, 4245, 3024, 3027, 5413, 5414, 3197, 319...","[4244_11296_11362, 4245_8613_8628, 3024_633_65...",3,1369,4655
34,methodological entity,data analysis method,autoregressive moving average model,0.0,[autoregressive moving average model],"[3694, 42, 6558, 6419, 2582, 5351, 3156]","[3694_5319_5342, 3694_5342_5362, 3694_5766_578...",1,7,46


In [84]:
#flatten list of entities which need to be removed 
entities_to_remove_cmpl = [x for xs in entities_to_removed.ent_list.to_list() for x in xs] # add all parent (and child) entities to a list which then can be excluded from the train dataframe

In [85]:
# Check number of entities which need to be removed from train set
len(entities_to_remove_cmpl)

168

In [87]:
# Store final entities_to_remove df as .csv
#entities_to_removed.to_csv(data_path_group+"ent_to_remove_from_ontology_final.csv")

### Create train dataframe of ontology

In [88]:
# Create train dataframe which does NOT contain any of the entities and porential children
entities_train = entities[~entities.ent_id.isin(entities_to_remove_cmpl)]
entities_train.shape

(1716198, 13)

In [89]:
# Store train df as .csv
#entities_train.to_csv(data_path_group+"ontology_train.csv")

### Create validation dataframe of ontology

For the validation and test set we need to further split the `entities_to_removed` dataframe

In [90]:
entities_to_removed.groupby(["level_1", "level_2"])[["num_ent"]].aggregate(["sum"])

num_ent
                                                sum
level_1                level_2                     
domain specific entity Economic sector           13
                       IS technology             38
                       IS topic                  35
                       study object               6
methodological entity  data analysis method      23
                       research method           19
                       sampling                   2
                       statistical metric         2
                       study object               1
                       validity                   4
theoretical entity     level of analysis          4
                       model                      5
                       research paradigm          3
                       sampling                   1
                       theory                    12